# Fine-tuning de GPT-2 com o Dataset AmazonTitles-1.3MM usando Hugging Face

Este notebook demonstra o processo de fine-tuning do modelo GPT-2 usando o dataset AmazonTitles-1.3MM. Utilizaremos as bibliotecas Hugging Face Datasets e Transformers para este processo.

## 0. Instalação das Dependências

Primeiro, vamos instalar as bibliotecas necessárias para o projeto.

In [ ]:
!pip install transformers datasets torch evaluate

## 1. Importação das Bibliotecas Necessárias

In [ ]:
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

## 2. Carregamento e Preparação do Dataset

In [ ]:
# Carregando o dataset
# Nota: Você precisará substituir 'path/to/your/dataset' pelo caminho real do seu dataset
dataset = load_dataset('json', data_files={'train': 'path/to/your/dataset/trn.json'})

# Função para preparar os dados
def prepare_data(examples):
    return {
        'text': [f"Title: {title}\nContent: {content}" for title, content in zip(examples['title'], examples['content'])]
    }

# Aplicando a preparação dos dados
dataset = dataset.map(prepare_data, batched=True, remove_columns=dataset['train'].column_names)

# Dividindo o dataset em treino e validação
dataset = dataset['train'].train_test_split(test_size=0.1)

print(f"Tamanho do conjunto de treinamento: {len(dataset['train'])}")
print(f"Tamanho do conjunto de validação: {len(dataset['test'])}")

## 3. Configuração do Modelo e Tokenizador

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Configurando o tokenizador
tokenizer.pad_token = tokenizer.eos_token

# Função para tokenizar os dados
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Aplicando a tokenização
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Preparando o data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## 4. Configuração do Treinamento

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

## 5. Treinamento do Modelo

In [ ]:
trainer.train()

## 6. Salvando o Modelo

In [ ]:
trainer.save_model('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

# Para baixar o modelo treinado do Colab
from google.colab import files
!zip -r fine_tuned_gpt2.zip fine_tuned_gpt2
files.download('fine_tuned_gpt2.zip')

## 7. Função para Gerar Respostas

In [ ]:
def generate_response(title, content):
    input_text = f"Title: {title}\nContent: {content}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Response:')[-1].strip()
    
    return response

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Título: {title}")
print(f"Conteúdo: {content}")
print(f"Resposta gerada: {response}")

## Conclusão

Este notebook demonstra o processo de fine-tuning do modelo GPT-2 usando o dataset AmazonTitles-1.3MM com as bibliotecas Hugging Face. O modelo treinado pode ser usado para gerar respostas baseadas em títulos e descrições de produtos.

Lembre-se de que este é um exemplo simplificado e pode ser aprimorado de várias maneiras:

1. Ajuste de hiperparâmetros para melhorar o desempenho.
2. Implementação de técnicas de regularização para evitar overfitting.
3. Uso de GPU para acelerar o treinamento (já habilitado no Colab).
4. Implementação de early stopping e learning rate scheduling.
5. Experimentação com diferentes tamanhos de modelo GPT-2 (small, medium, large).

Para um projeto real, considere também implementar uma avaliação mais robusta do modelo e adaptar o código conforme necessário para atender aos requisitos específicos do projeto.